# **CODERS OF AMINGAON**

In [ ]:
%%capture
!pip install crewai
!pip install crewai_tools
!pip install 'crewai[tools]'

from crewai import Agent, Task, Crew
from crewai import LLM

!pip install langchain-groq langchain --quiet
!pip install requests --quiet

import os
from langchain_groq import ChatGroq
from crewai import Process

import pickle
import requests
import re
import time

In [ ]:
os.environ['GROQ_API_KEY'] = "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"

In [ ]:
# Initialize the ChatGroq LLM
groq_llm = ChatGroq(
    model="groq/deepseek-r1-distill-llama-70b",
    api_key=os.getenv("GROQ_API_KEY")
)

groq_llm1 = ChatGroq(
    model="groq/llama-3.3-70b-versatile",
    api_key=os.getenv("GROQ_API_KEY")
)

groq_llm2 = ChatGroq(
    model="groq/deepseek-r1-distill-qwen-32b",
    api_key=os.getenv("GROQ_API_KEY")
)

groq_llm3 = ChatGroq(
    model="groq/gemma2-9b-it",
    api_key=os.getenv("GROQ_API_KEY")
)

groq_llm4 = ChatGroq(
    model="groq/llama-3.1-8b-instant",
    api_key=os.getenv("GROQ_API_KEY")
)

**Base story generation**

In [ ]:
from crewai import Agent, Task, Crew

class StoryGeneratorCrew:
    def __init__(self):
        self.groq_llm2 = groq_llm2
        self.groq_llm = groq_llm
        self.groq_llm1 = groq_llm1

    def story_and_char_agent(self):
        return Agent(
            role="Story & Character Arc Generator",
            goal="""
                You are a master storyteller. Your task is to take a brief prompt from a user and expand it into a 1500-word immersive story that is rich in detail and world-building.
                Make sure that using this idea hundreds of pages can be written and have many different characters.
                Additionally, generate well-defined character arcs for all major characters, ensuring they have depth, motivations, and transformations over time.

                The generated story should:
                - Be engaging, with clear conflict, tension, and resolution.
                - Allow for further expansion (open-ended elements, world-building hooks).
                - Maintain consistency across characters and settings.
                - Use as many characters as possible (atleast 6)

                Character arcs should include:
                - Background & Personality
                - Strengths & Flaws
                - Initial State → Transformation → Final State

                **Output Format:**
                ```
                Story:
                ---------------------------
                [1500-word immersive narrative]

                Characters:
                ---------------------------
                Character 1:
                - Name: [Character Name]
                - Background: [Where they come from, past experiences]
                - Personality: [Key traits, defining behaviors]
                - Strengths: [Unique skills, abilities]
                - Flaws: [Weaknesses, internal conflicts]
                - Arc Progression: [How they change over time]

                Character 2:
                [Same structure as Character 1]


                MAXIMUM 7 MAIN CHARACTERS
                ...
                ```
            """,
            backstory="""
                As a Story & Arc Agent, your mission is to craft compelling narratives and character journeys based on minimal user input.
                You ensure deep storytelling, character consistency, and an engaging, expandable world.
            """,
            verbose=True,
            llm=self.groq_llm2,
            max_iter=2,
        )

    def story_and_char_task(self, agent, input_data):
        return Task(
            description=f"""
                Generate a detailed, immersive 1500-word story and well-defined character arcs based on the provided prompt.

                Prompt:
                ```
                {input_data}
                ```
                Guidelines:
                - Expand the prompt into a structured narrative with world-building and compelling conflict.
                - Ensure character consistency, depth, and transformation over time.
                - Keep the story open-ended for further elaboration.
                - Use as many characters as possible (atleast 6)

                Expected Output Format:
                ```
                Story:
                ---------------------------
                [Generated 1500-word narrative]

                Characters:
                ---------------------------
                Character 1:
                - Name: [Character Name]
                - Background: [Where they come from, past experiences]
                - Personality: [Key traits, defining behaviors]
                - Strengths: [Unique skills, abilities]
                - Flaws: [Weaknesses, internal conflicts]
                - Arc Progression: [How they change over time]

                Character 2:
                [Same structure as Character 1]

                MAXIMUM 7 MAIN CHARACTERS
                ...
                ```
            """,
            agent=agent,
            expected_output="[Generated story and character arcs]",
            async_execution=False,
        )

    def arc_structuring_agent(self):
        return Agent(
            role="Story Arc Structurer",
            goal="""
                Your job is to take a full-length story with character development and break it down into multiple structured arcs.

                Each arc should:
                - Be clearly defined with a name/title.
                - Have a brief description explaining key events.
                - Maintain continuity and logical progression.
                - Ensure narrative tension, stakes, and character evolution.

                **Output Format:**
                ```
                Arcs:
                ---------------------------
                Arc 1: [Arc Title]
                - Summary: [Brief description of key events]

                Arc 2: [Arc Title]
                - Summary: [Brief description of key events]

                Arc 3: [Arc Title]
                - Summary: [Brief description of key events]
                ...
                ```
            """,
            backstory="""
                As an Arc Structuring Agent, your role is to analyze a given story and divide it into meaningful narrative arcs.
                You ensure that the story remains engaging by structuring it into logical segments with clear transitions.
            """,
            verbose=True,
            llm=self.groq_llm1,
            max_iter=2,
        )

    def arc_structuring_task(self, agent, story):
        return Task(
            description=f"""
                Analyze the given story and character development, then break it into multiple structured arcs.

                **Input Data:**
                ```
                Story and character:
                {story}

                ```

                **Guidelines:**
                - Identify key arcs within the story.
                - Clearly name each arc and briefly describe what happens.
                - Ensure the arcs maintain a logical sequence.
                - Try to create as many meaningful arcs as possible.

                **Expected Output:**
                ```
                Arcs:
                ---------------------------
                Arc 1: [Arc Title]
                - Summary: [Brief description of key events]

                Arc 2: [Arc Title]
                - Summary: [Brief description of key events]

                Arc 3: [Arc Title]
                - Summary: [Brief description of key events]
                ...
                ```
            """,
            agent=agent,
            expected_output="[Generated structured arcs]",
            async_execution=False,
        )

# Instantiate and Run
story_crew = StoryGeneratorCrew()
agent1 = story_crew.story_and_char_agent()
task1 = story_crew.story_and_char_task(agent1, "A story about a middle class indian boy going from rags to riches")
crew1 = Crew(agents=[agent1],
             tasks=[task1],
             verbose=False,
              max_rpm=29,)

results1 = crew1.kickoff()

agent2 = story_crew.arc_structuring_agent()
task2 = story_crew.arc_structuring_task(agent2, results1)
crew2 = Crew(agents=[agent2],
             tasks=[task2],
             verbose=False,
              max_rpm=29,)

results2 = crew2.kickoff()


# Agent: Story & Character Arc Generator
## Task: 
                Generate a detailed, immersive 1500-word story and well-defined character arcs based on the provided prompt.

                Prompt:
                ```
                A story about a middle class indian boy going from rags to riches
                ```
                Guidelines:
                - Expand the prompt into a structured narrative with world-building and compelling conflict.
                - Ensure character consistency, depth, and transformation over time.
                - Keep the story open-ended for further elaboration.
                - Use as many characters as possible (atleast 6)

                Expected Output Format:
                ```
                Story:
                ---------------------------
                [Generated 1500-word narrative]

                Characters:
                ---------------------------
                Character 1:
                - Name: [Character Name]
  

In [ ]:
import re

def extract_arcs_from_crew(crew_results):
    """
    Extracts story arcs from a CrewOutput object and returns them as a structured list.

    :param crew_results: The generated CrewOutput from Crew.kickoff()
    :return: A list of extracted arcs.
    """

    if hasattr(crew_results, "raw"):
        crew_text = crew_results.raw.strip()
    else:
        raise ValueError("CrewOutput object does not contain 'raw' output.")

    arc_pattern = r"(Arc \d+:.*?)$(?=\nArc \d+:|\Z)"
    arcs = re.findall(arc_pattern, crew_text, re.DOTALL | re.MULTILINE)

    return [arc.strip() for arc in arcs]

arc_list = extract_arcs_from_crew(results2)


In [ ]:
arc_list

['Arc 1: Introduction to Nainpur\n- Summary: The story begins in the rural village of Nainpur, where we meet the protagonist, Arjun Patel, and his family. We see the struggles of rural poverty and the determination of Arjun to make a better life for himself and his family.',
 'Arc 2: The Move to Mumbai\n- Summary: The Patel family is forced to move to Mumbai after a severe flood destroys their crops. They face the harsh realities of urban poverty, but Arjun remains determined to succeed. He starts working as a part-time assistant at a local grocery store and continues his education.',
 "Arc 3: The Birth of a Business\n- Summary: Arjun meets Rohan, a street-smart teenager, and they start a small business selling snacks and beverages to workers at a nearby construction site. Their venture grows, and they rent a small shop in the market area, marking the beginning of Arjun's entrepreneurial journey.",
 "Arc 4: Partnership and Growth\n- Summary: Arjun's hard work catches the attention of M

In [ ]:
results1

CrewOutput(raw='**Story:**  \n**Title: "From the Soil to the Skies"**  \n\nIn the heart of rural India, nestled between golden fields of wheat and millet, lay the modest village of Nainpur. It was here that our protagonist, Arjun Patel, was born into a family of modest means. His parents, Rajiv and Leela Patel, worked tirelessly as farmers, struggling to make ends meet while raising their three children: Arjun, his older sister, Anika, and younger brother, Rohan.  \n\nFrom a young age, Arjun exhibited a determination that set him apart. He was always the first to wake up, helping his father with the morning chores before rushing off to the village school. His teachers marveled at his insatiable curiosity and relentless pursuit of knowledge. But life in Nainpur was hard, and the Patels often went to bed with empty stomachs, hoping for a brighter tomorrow.  \n\nOne fateful monsoon season, tragedy struck. A severe flood destroyed the family\'s crops, leaving them in utter destitution. Wit

In [ ]:
story_data = results1.raw.split("\n\n**Characters:**  \n")[0]
characters_data = results1.raw.split("\n\n**Characters:**  \n")[1]

In [ ]:
story_data

'**Story:**  \n**Title: "From the Soil to the Skies"**  \n\nIn the heart of rural India, nestled between golden fields of wheat and millet, lay the modest village of Nainpur. It was here that our protagonist, Arjun Patel, was born into a family of modest means. His parents, Rajiv and Leela Patel, worked tirelessly as farmers, struggling to make ends meet while raising their three children: Arjun, his older sister, Anika, and younger brother, Rohan.  \n\nFrom a young age, Arjun exhibited a determination that set him apart. He was always the first to wake up, helping his father with the morning chores before rushing off to the village school. His teachers marveled at his insatiable curiosity and relentless pursuit of knowledge. But life in Nainpur was hard, and the Patels often went to bed with empty stomachs, hoping for a brighter tomorrow.  \n\nOne fateful monsoon season, tragedy struck. A severe flood destroyed the family\'s crops, leaving them in utter destitution. With no other opti

In [ ]:
characters_data

'\n**1. Arjun Patel**  \n- **Background:** Born into a poor farming family in Nainpur, Arjun grew up witnessing the struggles of rural poverty. His family\'s move to Mumbai during his teenage years exposed him to the harsh realities of urban life.  \n- **Personality:** Determined, resourceful, and compassionate. Arjun is driven by a deep sense of responsibility towards his family and community.  \n- **Strengths:** Entrepreneurial skills, leadership, and unwavering integrity.  \n- **Flaws:** Initially naive and idealistic, Arjun struggles with balancing his ambition with his moral compass.  \n- **Arc Progression:** Arjun begins as a shy, hardworking boy from the village, transforms into a successful businessman, and ultimately becomes a community leader who prioritizes social responsibility over personal gain.  \n\n**2. Rajiv Patel**  \n- **Background:** A former farmer turned laborer, Rajiv has spent his life struggling to provide for his family. His battles with alcoholism due to the 

**Base story continuation**

In [ ]:
def story_continuation_agent():
    return Agent(
        role="Story Continuation & Coherence Agent",
        goal="""
            You are a skilled storyteller responsible for ensuring seamless story progression.
            Your task is to expand upon a given story arc (arc_list[0]) while maintaining consistency with the overall story (story_data) and ensuring coherence with the following arc (arc_list[1]).

            **Your output should:**
            - Develop a highly detailed and immersive narrative relevant to the given arc.(min word limit of 1000 words)
            - Ensure a smooth connection between the given arc and the next one, without directly revealing future events.
            - Do not go beyond the given arc. Use story context only for referenece.
            - Maintain character consistency as per characters_data.
            - Include engaging dialogues (both narration and character speech) in a standard format.
            - Build tension, conflicts, and resolutions to keep the story compelling.

            **Expected Output Format:**
            ```
            Story Continuation:
            ---------------------------
            [Detailed, engaging story with dialogues in standard format]
            ```
        """,
        backstory="""
            As a Story Continuation & Coherence Agent, your mission is to extend a given story arc while ensuring logical flow and engagement.
            You maintain narrative depth, coherence, and character consistency.
        """,
        verbose=True,
        llm=groq_llm1,
        max_iter=2,
    )

def story_continuation_task(agent, story_data, arc_list0, arc_list1, characters_data):
    return Task(
        description=f"""
            Expand upon the current story arc while ensuring coherence with the overall story and the next arc.

            **Input Data:**
            - Overall Story: {story_data}
            - Current Arc: {arc_list0}
            - Next Arc (for coherence): {arc_list1}
            - Character Information: {characters_data}

            **Guidelines:**
            - Ensure the story stays true to existing elements and characters.
            - Write a detailed and immersive narrative.
            - Incorporate dialogues (narration + character speech) in a standard format.
            - Maintain a logical and engaging transition between arcs.

            **Expected Output Format:**
            ```
            Story Continuation:
            ---------------------------
            [Generated immersive story with dialogues]
            ```
        """,
        agent=agent,
        expected_output="[Generated story segment with coherent transitions and dialogues]",
        async_execution=False,
    )


In [ ]:
agent3 = story_continuation_agent()
task3 = story_continuation_task(agent3, story_data, arc_list[0], arc_list[1], characters_data)
crew3 = Crew(agents=[agent3],
             tasks=[task3],
             verbose=False,
              max_rpm=29,)

results3 = crew3.kickoff()

# Agent: Story Continuation & Coherence Agent
## Task: 
            Expand upon the current story arc while ensuring coherence with the overall story and the next arc.

            **Input Data:**
            - Overall Story: **Story:**  
**Title: "From the Soil to the Skies"**  

In the heart of rural India, nestled between golden fields of wheat and millet, lay the modest village of Nainpur. It was here that our protagonist, Arjun Patel, was born into a family of modest means. His parents, Rajiv and Leela Patel, worked tirelessly as farmers, struggling to make ends meet while raising their three children: Arjun, his older sister, Anika, and younger brother, Rohan.  

From a young age, Arjun exhibited a determination that set him apart. He was always the first to wake up, helping his father with the morning chores before rushing off to the village school. His teachers marveled at his insatiable curiosity and relentless pursuit of knowledge. But life in Nainpur was hard, and the Patels

In [ ]:
results3

CrewOutput(raw='Story Continuation:\n---------------------------\nThe sun had just begun to set over the small village of Nainpur, casting a warm orange glow over the thatched roofs and the bustling town square. Arjun Patel, a young boy with a determined look in his eyes, was busy helping his father, Rajiv, with the evening chores. They were a family of modest means, living off the land and struggling to make ends meet. But despite the hardships, Arjun\'s spirit remained unbroken.\n\n"Arjun, beta, can you help me with the cows?" Rajiv asked, his voice weary from a long day of work.\n\n"Yes, Papa," Arjun replied, quickly finishing his task and rushing to his father\'s side. Together, they led the cows to the nearby fields, where they would graze for the night.\n\nAs they walked, Rajiv put a hand on Arjun\'s shoulder. "You know, beta, I\'m proud of you. You\'re a hard worker, just like your mother. She\'s always been the strong one in our family, but I think you\'re going to surpass us b

In [ ]:
import re

def extract_story(text):
    pattern = r"Story Continuation:\n[-]+\n(.*)"
    match = re.search(pattern, text, re.DOTALL)

    if match:
        return match.group(1).strip()
    return "No story found."

raw_text = results3.raw
story = extract_story(raw_text)
print(story)


The sun had just begun to set over the small village of Nainpur, casting a warm orange glow over the thatched roofs and the bustling town square. Arjun Patel, a young boy with a determined look in his eyes, was busy helping his father, Rajiv, with the evening chores. They were a family of modest means, living off the land and struggling to make ends meet. But despite the hardships, Arjun's spirit remained unbroken.

"Arjun, beta, can you help me with the cows?" Rajiv asked, his voice weary from a long day of work.

"Yes, Papa," Arjun replied, quickly finishing his task and rushing to his father's side. Together, they led the cows to the nearby fields, where they would graze for the night.

As they walked, Rajiv put a hand on Arjun's shoulder. "You know, beta, I'm proud of you. You're a hard worker, just like your mother. She's always been the strong one in our family, but I think you're going to surpass us both."

Arjun looked up at his father, a smile spreading across his face. "Thank

In [ ]:
continue_story = results3.raw.split(" pydantic=None, json_dict=None, tasks_output=[TaskOutput(description='\n            ")[0]

In [ ]:
continue_story

'Story Continuation:\n---------------------------\nThe sun had just begun to set over the small village of Nainpur, casting a warm orange glow over the thatched roofs and the bustling town square. Arjun Patel, a young boy with a determined look in his eyes, was busy helping his father, Rajiv, with the evening chores. They were a family of modest means, living off the land and struggling to make ends meet. But despite the hardships, Arjun\'s spirit remained unbroken.\n\n"Arjun, beta, can you help me with the cows?" Rajiv asked, his voice weary from a long day of work.\n\n"Yes, Papa," Arjun replied, quickly finishing his task and rushing to his father\'s side. Together, they led the cows to the nearby fields, where they would graze for the night.\n\nAs they walked, Rajiv put a hand on Arjun\'s shoulder. "You know, beta, I\'m proud of you. You\'re a hard worker, just like your mother. She\'s always been the strong one in our family, but I think you\'re going to surpass us both."\n\nArjun 

**QNA Agent**

In [ ]:
def story_question_agent():
    return Agent(
        role="Story Coherence Question Generator",
        goal="""
            Your task is to analyze a given story arc and generate relevant questions
            to ensure consistency, character integrity, and logical storytelling.

            **Your tasks include:**
            - Understanding the given story arc in depth.
            - Identifying key characters, events, and settings.
            - Generating detailed questions to verify coherence.
            - Ensuring the generated questions cover all aspects necessary for consistency.
            - Structuring the questions logically to guide the refinement process.

            **Expected Output Format:**
            ```
            Generated Questions:
            ---------------------------
            1. [Question about character consistency]
            2. [Question about event continuity]
            3. [Question about setting accuracy]
            ...
            ```
        """,
        backstory="""
            As a Story Coherence Question Generator, your role is to ensure that each story arc
            remains logically consistent and well-structured by formulating comprehensive questions.
        """,
        verbose=True,
        llm=groq_llm3,
        max_iter=3,
    )

def story_question_task(agent, continue_story):
    return Task(
        description=f"""
            Generate a list of questions based on the given story arc to ensure consistency.

            **Input Data:**
            - Continue Story: {continue_story}

            **Guidelines:**
            - Identify major characters, events, and settings.
            - Formulate detailed questions to check for inconsistencies.
            - Ensure that the questions address all necessary aspects of coherence.
            - Make sure to generate as many questions as needed.

            **Steps to Ensure Coherence:**
            1. Extract relevant character and event details from the continue story.
            2. Construct precise queries to verify story accuracy and coherence.
            3. Ensure questions are specific enough to resolve potential inconsistencies.
            4. Iterate up to 3 times to refine the questions if needed.

            **Expected Output Format:**
            ```
            Generated Questions:
            ---------------------------
            1. [Question about character backstory]
            2. [Question about relationship dynamics]
            3. [Question about timeline accuracy]
            ...
            ```
        """,
        agent=agent,
        expected_output="[List of well-structured questions ensuring story coherence]",
        async_execution=False,
    )



In [ ]:
# Kickoff
question_agent = story_question_agent()
question_task = story_question_task(question_agent, continue_story)

question_crew = Crew(agents=[question_agent],
                     tasks=[question_task],
                     verbose=False,
                     max_rpm=29,)

question_results = question_crew.kickoff()

# Agent: Story Coherence Question Generator
## Task: 
            Generate a list of questions based on the given story arc to ensure consistency.

            **Input Data:**
            - Continue Story: Story Continuation:
---------------------------
The sun had just begun to set over the small village of Nainpur, casting a warm orange glow over the thatched roofs and the bustling town square. Arjun Patel, a young boy with a determined look in his eyes, was busy helping his father, Rajiv, with the evening chores. They were a family of modest means, living off the land and struggling to make ends meet. But despite the hardships, Arjun's spirit remained unbroken.

"Arjun, beta, can you help me with the cows?" Rajiv asked, his voice weary from a long day of work.

"Yes, Papa," Arjun replied, quickly finishing his task and rushing to his father's side. Together, they led the cows to the nearby fields, where they would graze for the night.

As they walked, Rajiv put a hand on Arjun's sho

In [ ]:
question_results

CrewOutput(raw='```\nGenerated Questions:\n---------------------------\n1.  Since Arjun\'s family is struggling financially due to crop failure and drought, how are they able to afford the journey to Mumbai?\n2.  The story mentions Arjun\'s sister Anika.  What is her age and how does she contribute to the family\'s income?  \n3.  Arjun and Rohan start a snack and beverage business. How does their venture grow so quickly, and what resources or connections do they utilize to achieve this success?\n4.  Arjun\'s business flourishes over several years.  What specific challenges does he face during this time, and how does he overcome them?\n5.  The story states that Arjun remains "humble, kind, and generous." What specific actions or decisions demonstrate these qualities throughout his journey in Mumbai? \n6.  How does Arjun\'s upbringing in Nainpur influence his values and approach to success in the city?\n7.  The story emphasizes Arjun\'s determination and resilience. Can you pinpoint spec

In [ ]:
question_list = question_results.raw.split("Generated Questions:\n---------------------------\n")[1]
question_list

'1.  Since Arjun\'s family is struggling financially due to crop failure and drought, how are they able to afford the journey to Mumbai?\n2.  The story mentions Arjun\'s sister Anika.  What is her age and how does she contribute to the family\'s income?  \n3.  Arjun and Rohan start a snack and beverage business. How does their venture grow so quickly, and what resources or connections do they utilize to achieve this success?\n4.  Arjun\'s business flourishes over several years.  What specific challenges does he face during this time, and how does he overcome them?\n5.  The story states that Arjun remains "humble, kind, and generous." What specific actions or decisions demonstrate these qualities throughout his journey in Mumbai? \n6.  How does Arjun\'s upbringing in Nainpur influence his values and approach to success in the city?\n7.  The story emphasizes Arjun\'s determination and resilience. Can you pinpoint specific instances where his character traits are tested and how he demonst

In [ ]:
questions = re.findall(r'\d+\.\s(.+)', question_list)

# The questions array now contains all 20 questions
print(questions)
print(f"Total questions extracted: {len(questions)}")

[" Since Arjun's family is struggling financially due to crop failure and drought, how are they able to afford the journey to Mumbai?", " The story mentions Arjun's sister Anika.  What is her age and how does she contribute to the family's income?  ", ' Arjun and Rohan start a snack and beverage business. How does their venture grow so quickly, and what resources or connections do they utilize to achieve this success?', " Arjun's business flourishes over several years.  What specific challenges does he face during this time, and how does he overcome them?", ' The story states that Arjun remains "humble, kind, and generous." What specific actions or decisions demonstrate these qualities throughout his journey in Mumbai? ', " How does Arjun's upbringing in Nainpur influence his values and approach to success in the city?", " The story emphasizes Arjun's determination and resilience. Can you pinpoint specific instances where his character traits are tested and how he demonstrates these qu

In [ ]:
!pip install langchain langchain_groq faiss-cpu sentence-transformers --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 64.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not curre

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

loader = TextLoader("/content/rag data.txt")
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)
docs = text_splitter.split_documents(documents)
hf_embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(docs, hf_embedder)
retriever = vectorstore.as_retriever()

<ipython-input-23-4d2e8c19c26b>:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import os
from langchain_groq import ChatGroq

os.environ["GROQ_API_KEY"] = "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"

if not os.getenv("GROQ_API_KEY"):
    raise ValueError("GROQ_API_KEY is missing!")

llm = ChatGroq(
    model="gemma2-9b-it",
    api_key=os.getenv("GROQ_API_KEY")
)


In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff"
)


In [ ]:
questions[1]

" The story mentions Arjun's sister Anika.  What is her age and how does she contribute to the family's income?  "

In [ ]:
answers= []

In [ ]:
for i in range(len(questions)):
    response = qa_chain.run(questions[i])
    answers.append(response)



<ipython-input-28-1716018f97c9>:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(questions[i])


In [ ]:
answers[1]

"The provided text doesn't say how old Anika is.  \n\nIt does say that she works as a domestic help to contribute to the family's income. \n"

**Editor Agent**

In [ ]:
def story_editor_agent():
    return Agent(
        role="Story Arc Expansion & Enhancement Editor",
        goal="""
            Your task is to expand and enhance a given story arc while ensuring it logically fits into the broader story context.

            **Responsibilities:**
            - Deepen the provided arc with rich narrative and emotional development.
            - Ensure continuity with the broader story and character development.
            - Improve storytelling flow, pacing, and engagement within the arc.
            - Maintain essential character traits, motivations, and plot relevance.

            **Expected Output Format:**
            ```
            Expanded Arc:
            ---------------------------
            [Expanded and enhanced version of the arc, with added depth, tension, and narrative progression]
            ```
        """,
        backstory="""
            As an Arc Expansion & Enhancement Editor, your role is to take a brief or undeveloped arc and build it into a full, engaging sequence that aligns with the overall story.
            You enrich plotlines, character choices, and turning points, without straying from established story facts.
        """,
        verbose=True,
        llm=groq_llm1,
        max_iter=2,
    )

def story_editing_task(agent, story, arc_list, questions, answers):
    return Task(
        description=f"""
            Expand the given arc into a detailed, engaging narrative sequence that fits smoothly into the broader story.

            **Input Data:**
            - **Full Story Context:** {story}
            - **Arc to Expand:** {arc_list}
            - **Coherence Validation:** Use {questions} and {answers} to ensure logical and thematic consistency.

            **Arc Expansion Guidelines:**
            - Focus on expanding **only** the arc (not the full story).
            - Build depth into character interactions, conflicts, and emotional turning points within the arc.
            - Ensure expanded arc fits naturally with the broader story structure and timeline.
            - Use Q&A validation to verify that no contradictions or continuity errors are introduced.
            - Maintain key story themes, tones, and character voices.

            **Output Format:**
            ```
            Expanded Arc:
            ---------------------------
            [Expanded and detailed arc with smooth integration into the story]
            ```
        """,
        agent=agent,
        expected_output="[Expanded and enhanced story arc with consistent tone, logic, and emotion]",
        async_execution=False,
    )


In [ ]:

editor_agent = story_editor_agent()

editing_task = story_editing_task(
    editor_agent, story, arc_list[0], questions, answers
)

editing_crew = Crew(
    agents=[editor_agent],
    tasks=[editing_task],
    verbose=False,
    max_rpm=29,
)

editing_results = editing_crew.kickoff()


# Agent: Story Arc Expansion & Enhancement Editor
## Task: 
            Expand the given arc into a detailed, engaging narrative sequence that fits smoothly into the broader story.

            **Input Data:**
            - **Full Story Context:** The sun had just begun to set over the small village of Nainpur, casting a warm orange glow over the thatched roofs and the bustling town square. Arjun Patel, a young boy with a determined look in his eyes, was busy helping his father, Rajiv, with the evening chores. They were a family of modest means, living off the land and struggling to make ends meet. But despite the hardships, Arjun's spirit remained unbroken.

"Arjun, beta, can you help me with the cows?" Rajiv asked, his voice weary from a long day of work.

"Yes, Papa," Arjun replied, quickly finishing his task and rushing to his father's side. Together, they led the cows to the nearby fields, where they would graze for the night.

As they walked, Rajiv put a hand on Arjun's shoulder.

In [ ]:
editing_results

CrewOutput(raw='Expanded Arc:\n---------------------------\nThe sun had just begun to set over the small village of Nainpur, casting a warm orange glow over the thatched roofs and the bustling town square. Arjun Patel, a young boy with a determined look in his eyes, was busy helping his father, Rajiv, with the evening chores. They were a family of modest means, living off the land and struggling to make ends meet. But despite the hardships, Arjun\'s spirit remained unbroken.\n\nAs they worked, Rajiv couldn\'t help but think about the struggles they faced. The crops had been failing, and the drought had been getting worse. He knew that they might have to look for work in the city, a thought that filled him with a mix of emotions. He was proud of Arjun, who had grown into a hard worker, just like his mother. Leela, Arjun\'s mother, was the strong one in the family, always managing to make ends meet, no matter what.\n\n"Arjun, beta, can you help me with the cows?" Rajiv asked, his voice w

In [ ]:
story_refined = editing_results.raw.split("pydantic=None, json_dict=None,")[0]

In [ ]:
story_refined

'Expanded Arc:\n---------------------------\nThe sun had just begun to set over the small village of Nainpur, casting a warm orange glow over the thatched roofs and the bustling town square. Arjun Patel, a young boy with a determined look in his eyes, was busy helping his father, Rajiv, with the evening chores. They were a family of modest means, living off the land and struggling to make ends meet. But despite the hardships, Arjun\'s spirit remained unbroken.\n\nAs they worked, Rajiv couldn\'t help but think about the struggles they faced. The crops had been failing, and the drought had been getting worse. He knew that they might have to look for work in the city, a thought that filled him with a mix of emotions. He was proud of Arjun, who had grown into a hard worker, just like his mother. Leela, Arjun\'s mother, was the strong one in the family, always managing to make ends meet, no matter what.\n\n"Arjun, beta, can you help me with the cows?" Rajiv asked, his voice weary from a lon

In [ ]:
def episode_breakdown_agent():
    return Agent(
        role="Serialized Story Episode Writer",
        goal="""
            Your task is to break down a full story into a minimum of 10 fully written, interconnected narrative episodes.

            **Responsibilities:**
            - Divide the story into distinct but sequentially connected episodes.
            - Write out each episode as a full narrative, not just a summary.
            - Ensure each episode flows naturally into the next.
            - Expand on characters, events, and emotional arcs.
            - Maintain continuity and deepen the story while preserving its essence.

            **Expected Output Format:**
            ```
            Episode Story:
            ---------------------------
            Episode 1: [Title]
            [Fully written episode - rich, engaging, minimum 500 words]

            Episode 2: [Title]
            [Fully written episode continuing the story - minimum 500 words]
            ...
            (at least 10 connected episodes)
            ```
        """,
        backstory="""
            As a Serialized Story Episode Writer, your job is to convert a full story into a set of richly detailed, coherent episodes — like serialized fiction or show scripts.
            You enhance character development, deepen the plot, and ensure narrative immersion across all episodes.
        """,
        verbose=True,
        llm=groq_llm4,
        max_iter=2,
    )

def episode_breakdown_task(agent, story):
    return Task(
        description=f"""
            Transform the given story into a series of fully developed, interconnected narrative episodes.

            **Input Data:**
            - **Full Story:** {story}

            **Episode Writing Guidelines:**
            - Write **at least 10 episodes**, each with a proper title and full narrative content.
            - Do **not** provide summaries — instead, write the **full narrative** for each episode.
            - Each episode should be **at least 500 words** and continue the story from the previous one.
            - Ensure continuity of characters, plotlines, and emotional development.
            - Add new scenes or subplots where appropriate to enhance storytelling depth.
            - Every episode should read like a chapter in a novel or script — immersive and complete.

            **Output Format:**
            ```
            Episode Story:
            ---------------------------
            Episode 1: [Title]
            [Complete story segment - minimum 500 words]

            Episode 2: [Title]
            [Next part of the story - minimum 500 words]
            ...
            Episode 10+: [Title]
            [Continued story - minimum 500 words]
            ```
        """,
        agent=agent,
        expected_output="[Complete story told over 10+ episodic parts, minimum 500 words per episode]",
        async_execution=False,
    )


In [ ]:
breakdown_agent = episode_breakdown_agent()
breakdown_task = episode_breakdown_task(
    breakdown_agent, story_refined
)

breakdown_crew = Crew(
    agents=[breakdown_agent],
    tasks=[breakdown_task],
    verbose=False,
    max_rpm=29,
)

episode_results = breakdown_crew.kickoff()


# Agent: Serialized Story Episode Writer
## Task: 
            Transform the given story into a series of fully developed, interconnected narrative episodes.

            **Input Data:**
            - **Full Story:** Expanded Arc:
---------------------------
The sun had just begun to set over the small village of Nainpur, casting a warm orange glow over the thatched roofs and the bustling town square. Arjun Patel, a young boy with a determined look in his eyes, was busy helping his father, Rajiv, with the evening chores. They were a family of modest means, living off the land and struggling to make ends meet. But despite the hardships, Arjun's spirit remained unbroken.

As they worked, Rajiv couldn't help but think about the struggles they faced. The crops had been failing, and the drought had been getting worse. He knew that they might have to look for work in the city, a thought that filled him with a mix of emotions. He was proud of Arjun, who had grown into a hard worker, just like